In [2]:
import sys
import json
import pandas as pd
import numpy as np
sys.path.insert(1, 'C:/Users/Admin/Projects/voice-anomaly-detector')

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, classification_report

from models.baseline import Baseline
from models.lightgbm_interface import LightGBMInterface

In [3]:
columns = ['Zero crossings', 'Centroid AUC', 'Rolloff AUC', 'Centroid mean',
       'Rolloff mean', 'Centroid STD', 'Rolloff STD', 'Centroid p-t-p value',
       'Rolloff p-t-p value', 'Centroid_percentile', 'Rolloff_percentile',
       'MFCC mean 1', 'MFCC mean 2', 'MFCC mean 3', 'MFCC mean 4',
       'MFCC mean 5', 'MFCC mean 6', 'MFCC mean 7', 'MFCC mean 8',
       'MFCC mean 9', 'MFCC mean 10', 'MFCC mean 11', 'MFCC mean 12',
       'MFCC mean 13', 'MFCC delta 1', 'MFCC delta 2', 'MFCC delta 3',
       'MFCC delta 4', 'MFCC delta 5', 'MFCC delta 6', 'MFCC delta 7',
       'MFCC delta 8', 'MFCC delta 9', 'MFCC delta 10', 'MFCC delta 11',
       'MFCC delta 12', 'MFCC delta 13', 'MFCC delta 2 1', 'MFCC delta 2 2',
       'MFCC delta 2 3', 'MFCC delta 2 4', 'MFCC delta 2 5', 'MFCC delta 2 6',
       'MFCC delta 2 7', 'MFCC delta 2 8', 'MFCC delta 2 9', 'MFCC delta 2 10',
       'MFCC delta 2 11', 'MFCC delta 2 12', 'MFCC delta 2 13']

In [4]:
def train_model(df, params):
    k_fold = StratifiedKFold(n_splits=5, shuffle=True)
    splits = k_fold.split(df, df["Label"])

    test_result = []
    train_result = []

    encode_labels = preprocessing.LabelEncoder()
    encode_labels.fit(df["Label"])
    labels = encode_labels.transform(df["Label"])

    for train_index, test_index in splits:

        df = df[columns]
        train_x = df.loc[train_index]
        test_x = df.loc[test_index]
        labels_train = labels[train_index]
        labels_test = labels[test_index]

        # model = Baseline()
        model = LogisticRegression(C=30, solver='newton-cg')
        # model = RandomForestClassifier(**params)
        # model = SVC(**params)
        # model = LightGBMInterface(params=params)

        model.fit(train_x, labels_train)
        y_predict_test = model.predict(test_x)
        y_predict_train = model.predict(train_x)
        score_test = f1_score(labels_test, y_predict_test)
        score_train = f1_score(labels_train, y_predict_train)
        print(classification_report(labels_test, y_predict_test))
        test_result.append(score_test)
        train_result.append(score_train)
    return test_result, train_result

In [16]:
if __name__ == '__main__':
    df = pd.read_csv('../data/features.csv')

    params = {'n_estimators': 100, 'max_depth': 30, 'min_samples_split': 50, 'min_samples_leaf': 20, 'max_samples': 100}
    # Random Forest params
    # params = {'num_iterations': 40, 'num_leaves': 50, 'min_data_in_leaf': 40, 'objective': 'binary'}  # LightGBM params
    # params = {'C': 5, 'kernel': 'sigmoid', 'gamma': 'auto'} # SVM params

    test_result, train_result = train_model(df, params)

    result_test_mean = np.mean(test_result)
    result_test_std = np.std(test_result)
    print(f'Test score: {result_test_mean}')
    print(f'STD: {result_test_std}')

    result_train_mean = np.mean(train_result)
    result_train_std = np.std(train_result)
    print('Train score: ', result_train_mean)
    print('STD: ', result_train_std)

c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\sklearn\utils\optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


              precision    recall  f1-score   support

           0       0.68      0.72      0.70       138
           1       0.70      0.66      0.68       137

    accuracy                           0.69       275
   macro avg       0.69      0.69      0.69       275
weighted avg       0.69      0.69      0.69       275



c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\v

              precision    recall  f1-score   support

           0       0.73      0.78      0.76       138
           1       0.76      0.71      0.73       137

    accuracy                           0.75       275
   macro avg       0.75      0.75      0.75       275
weighted avg       0.75      0.75      0.75       275



c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\sklearn\utils\optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


              precision    recall  f1-score   support

           0       0.64      0.74      0.69       137
           1       0.69      0.59      0.64       138

    accuracy                           0.67       275
   macro avg       0.67      0.67      0.66       275
weighted avg       0.67      0.67      0.66       275



c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\sklearn\utils\optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


              precision    recall  f1-score   support

           0       0.69      0.76      0.72       137
           1       0.74      0.67      0.70       138

    accuracy                           0.71       275
   macro avg       0.71      0.71      0.71       275
weighted avg       0.71      0.71      0.71       275

              precision    recall  f1-score   support

           0       0.74      0.74      0.74       137
           1       0.74      0.74      0.74       137

    accuracy                           0.74       274
   macro avg       0.74      0.74      0.74       274
weighted avg       0.74      0.74      0.74       274

Test score: 0.6988530559314421
STD: 0.03676486782060712
Train score:  0.7104437278383111
STD:  0.006847005269814255


c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\venv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\users\admin\projects\voice-anomaly-detector\v